<a href="https://colab.research.google.com/github/leochartrand/IFT615/blob/main/RNN/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Apprenons à un RNN à générer du Shakespeare!

*Mary, and will, my lord, to weep in such a one were prettiest​*

​*Yet now I was adopted heir​​*

​*Of the world’s lamentable day​​*

​*To watch the next way with his father with his face?*

Commençons par importer nos librairies

In [110]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
import random

Téléchargeons maintenant le jeu de données. Ce dernier est un ensemble de 40,000 lignes de Shakespeare provenant d'une variété de ses oeuvres. Ce *dataset* a été assemblé par Andrej Karpathy: https://github.com/karpathy/char-rnn (BSD License)

In [ ]:
!wget https://raw.githubusercontent.com/leochartrand/IFT615/main/RNN/shakespeare.txt

data = open(r'shakespeare.txt').read()

Jetons un coup d'oeil au dataset. On peut imprimer par exemple les 1000 premiers charactères:

In [ ]:
print(data[:1000])

Prétraitons le jeu de donner un peu plus. Commençons par créer une "dictionnaire" des charactères que l'on trouve dans le jeu de données:

In [ ]:
char_dict = sorted(list(set(data)))
vocab_size = len(char_dict)
print("Le jeu de données est constitué de %d charactères uniques."%vocab_size)

Il est important que les charactères puissent être interprétés par notre modèle. Pour cela, on peut commencer par convertir chaque charactère en un indice, qui prend position dans l'intervalle [0, vocab_size[. On veut aussi pouvoir faire l'inverse et obtenir des charactères à nouveau à la sortie du modèle:

In [50]:
# Pour éviter de faire la conversion plusieurs fois et de répéter du code, on
# peut créer des dictionnaires qui font un "mapping" entre les charactères et leur index
char_to_index = {char:i for i,char in enumerate(char_dict)}
index_to_char = {i:char for i,char in enumerate(char_dict)}

# Créons une simple fonction qui permettra d'obtenir les charactères directement lors de la génération:
def get_char(index):
  return index_to_char[index]
def get_index(char):
  return char_to_index[char]

# On peut déjà convertir notre jeu de données en une liste d'indices!
data = list(data)
for i, char in enumerate(data):
    data[i] = get_index(char)

On peut maintenant créer notre modèle!

In [92]:
class Char_RNN(nn.Module):
  def __init__(self, input_size, output_size, hidden_size, num_layers):
    super(Char_RNN, self).__init__()
    self.layers = num_layers
    self.hidden_size = hidden_size

    self.embedding = nn.Embedding(input_size, input_size)
    self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
    self.dense = nn.Linear(hidden_size, output_size)

  def forward(self, input, hidden):
    embedded = self.embedding(input)
    output, hidden = self.rnn(embedded, hidden)
    output = self.dense(output)
    return output, hidden

  def init_hidden(self, batch_size):
        return torch.zeros(self.layers, batch_size, self.hidden_size)

char_model = Char_RNN(vocab_size, vocab_size, 128, 3)

On passe à l'entraînement...

In [ ]:
# Définissons nos hyperparametres
epochs = 5
batch_size = 100
sequence_length = 50
learning_rate = 5e-4

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(char_model.parameters(), lr=learning_rate)

# On garde un historique de l'entraînement pour visualiser nos résultats
loss_history = []

for epoch in range(epochs):
  print(f"Epoch {epoch + 1}/{epochs}")
  total_batches = (len(data) - sequence_length) // (sequence_length * batch_size)
  with tqdm(total=total_batches) as progress:
   for _ in range(total_batches):
      start_index = random.randint(0, 100)

      if start_index + sequence_length * batch_size > len(data) - sequence_length:
          continue

      input_seq = torch.tensor([data[j:j+sequence_length] for j in range(start_index, start_index + sequence_length * batch_size, sequence_length)])
      target_seq = torch.tensor([data[j+1:j+sequence_length+1] for j in range(start_index, start_index + sequence_length * batch_size, sequence_length)])


      if input_seq.size(0) != batch_size:
          continue

      hidden = char_model.init_hidden(batch_size)
      char_model.zero_grad()

      output, hidden = char_model(input_seq, hidden)
      loss = criterion(output.view(-1, vocab_size), target_seq.view(-1))

      loss.backward()
      optimizer.step()

      loss_history.append(loss.item())
      progress.update(1)
      progress.set_description("Loss: %.4f"%loss.item())

Visualisons la courbe d'apprentissage:

In [ ]:
plt.plot(loss_history)
plt.xlabel('Batches')
plt.ylabel('Loss')
plt.title('Loss history')
plt.show()

Essayons maintenant degénérer du Shakespeare!

In [ ]:
seed = "Bob"
temperature = 0.9
gen_length = 2000

char_model.eval()

with torch.no_grad():
  for _ in range(gen_length):
    # On prépare les données et le modèle
    input_tensor = torch.tensor([[get_index(c) for c in seed][-sequence_length:]], dtype=torch.long)
    hidden = char_model.init_hidden(1)

    # Forward pass
    output, hidden = char_model(input_tensor, hidden)

    # On sélectionne le charactère selon les probabilités (sous forme d'indice)
    # Ici, une variable de température assure une certaine stochasticité
    logits = output[0, -1, :] / temperature
    # On normalise les probabilités avec un softmax
    prob_dist = F.softmax(logits, dim=0)

    # On échantillone ensuite un indice dans la distribution
    top_char_index = torch.multinomial(prob_dist, 1).item()

    # On reconvertit en charactère
    predicted_char = get_char(top_char_index)
    seed += predicted_char

    # Print the new character
    print(predicted_char, end='', flush=True)
